# First thing to do, make a copy of this notebook under File->Make a Copy with some intelligent naming convention.

---

# Set your observing times here.

## NB: This will be a broader window that you'll actually use. The final output will give you orbit-by-orbit pointing information. So you can use a subset of the orbits that are chosen here.

In [1]:
from datetime import datetime
import numpy as np
tstart = '2017-10-23T00:00:00'
tend = '2017-10-24T00:00:00'
 
# Turn these into datetime objects

tstart = datetime.strptime(tstart, '%Y-%m-%dT%H:%M:%S')
tend = datetime.strptime(tend, '%Y-%m-%dT%H:%M:%S')

#tstart.strftime('%Y:%j:%H:%M:%S')


---
# Step 1, get the nominal RA/Dec Position for the Moon at the start.

## We use this to determine what the occultation times are.

In [2]:
from skyfield.api import Loader
from astropy.time import Time
import astropy.units as u

load = Loader('../../data')

ts = load.timescale()
planets = load('de436.bsp')

astro_time = Time(tstart)
t = ts.from_astropy(astro_time)
moon, earth = planets['moon'], planets['earth']
astrometric = earth.at(t).observe(moon)
ra, dec, distance = astrometric.radec()


occstring = "./occ {0:0.4f} {1:0.4f} Latest_TLE.txt {2}:{3}:00:00:00 {4}:{5}:00:00:00 lunar_{3}_{5}.occ".format(
      ra.to(u.deg).value, dec.to(u.deg).value,
      tstart.timetuple().tm_year, tstart.timetuple().tm_yday, 
      tend.timetuple().tm_year, tend.timetuple().tm_yday)


outfile = '../orbit_engine/lunar_{0}to{1}.sh'.format(tstart.timetuple().tm_yday, tend.timetuple().tm_yday)
print(outfile)

f = open(outfile, 'w')
f.write(occstring)
f.close()


import os
import stat

st = os.stat(outfile)
os.chmod(outfile, st.st_mode | stat.S_IEXEC)



../orbit_engine/lunar_296to297.sh


---
# Step 2: Go run the code that figures out the unocculted periods for the RA/Dec and the date range reported above.

This works on lif. There are example shell scripts in the ../orbit_engine directory that use the version that Karl already compiled for the nuops users.

First, get the latest TLE archive:

`./get_latest_TLE.sh`

Run the script that was produced above.


# Step 3: Initialize your libraries and parse the resulting occultation file:


In [3]:
from nustar_planning import io
occfile= "../orbit_engine/lunar_{0}_{1}.occ".format(tstart.timetuple().tm_yday,tend.timetuple().tm_yday )

#print(occfile)
orbits = io.parse_occ(occfile)

# NB: The "head" command here only shows the first couple of rows. Do a "print(orbits)" to see them all.
orbits.head()


,visible,occulted
0,2017-10-23 00:38:39,2017-10-23 01:35:02
1,2017-10-23 02:15:19,2017-10-23 03:11:41
2,2017-10-23 03:51:58,2017-10-23 04:48:20
3,2017-10-23 05:28:38,2017-10-23 06:25:00
4,2017-10-23 07:05:17,2017-10-23 08:01:39


In [4]:
#from nustar_planning.lunar_planning import position, position_shift
import nustar_planning.lunar_planning as lunar_planning
from importlib import reload
reload(lunar_planning)

<module 'nustar_planning.lunar_planning' from '/Users/bwgref/science/local/anaconda/lib/python3.5/site-packages/nustar_planning-0.1.dev50-py3.5.egg/nustar_planning/lunar_planning.py'>

In [9]:
reload(lunar_planning)
sub = orbits.iloc[5:6].reset_index()
lunar_planning.position(sub, steps = 5, parallax_correction=True)

Aim Time            RA         Dec
2017-10-23T08:41:57 250.26203  -17.23396
2017-10-23T08:53:13.200000 250.32626  -17.49259
2017-10-23T09:04:29.400000 249.89779  -17.64254
2017-10-23T09:15:45.600000 249.24485  -17.60832
2017-10-23T09:27:01.800000 248.76562  -17.41580


In [11]:
# PA == 0 (default), Rmoon = 940 arcseconds (default), 6 arcminutes maximum drift (default) and
# no minimum dwell period (default)
reload(lunar_planning)
lunar_planning.position_shift(sub, parallax_correction=True, pa = 338.0*u.deg, dt = 3*u.s,
                              outfile='lunar_20171023_v3.txt')

2017:296:08:36:57 RA: 250.26174  Dec: -17.40715

2017:296:08:39:03 RA: 250.34909  Dec: -17.46402

2017:296:08:41:27 RA: 250.42477  Dec: -17.53533

2017:296:08:44:24 RA: 250.46783  Dec: -17.62693

2017:296:08:48:15 RA: 250.43858  Dec: -17.72324

2017:296:08:52:39 RA: 250.35823  Dec: -17.78862

2017:296:08:56:09 RA: 250.26120  Dec: -17.83125

2017:296:08:58:57 RA: 250.16033  Dec: -17.85924

2017:296:09:01:18 RA: 250.05506  Dec: -17.87802

2017:296:09:03:27 RA: 249.94821  Dec: -17.88929

2017:296:09:05:27 RA: 249.84076  Dec: -17.89429

2017:296:09:07:21 RA: 249.73541  Dec: -17.89386

2017:296:09:09:09 RA: 249.62838  Dec: -17.88842

2017:296:09:10:57 RA: 249.52142  Dec: -17.87809

2017:296:09:12:45 RA: 249.41627  Dec: -17.86305

2017:296:09:14:33 RA: 249.31190  Dec: -17.84295

2017:296:09:16:24 RA: 249.20793  Dec: -17.81708

2017:296:09:18:21 RA: 249.10745  Dec: -17.78522

2017:296:09:20:24 RA: 249.01008  Dec: -17.74554

2017:296:09:22:39 RA: 248.91754  Dec: -17.69483

2017:296:09:25:15 RA